In [ ]:
!pip install pandas 
!pip install psycopg2-binary
!pip install sqlalchemy

In [ ]:
import json
from kafka import KafkaConsumer
import psycopg2
from psycopg2.extras import execute_values

# PostgreSQL database connection details
db_config = {
    'dbname': 'mydb',
    'user': 'exampleuser',
    'password': 'abc',
    'host': '10.100.107.105',
    'port': 5432
}

# Kafka topic
postgres_topic = "postgresql"
brokers = ["ec2-54-80-241-235.compute-1.amazonaws.com:9092"]

# Create a Kafka consumer
consumer = KafkaConsumer(
    postgres_topic,
    bootstrap_servers=brokers,
    value_deserializer=lambda message: json.loads(message.decode('utf-8'))
)

# Connect to PostgreSQL
try:
    conn = psycopg2.connect(**db_config)
    cursor = conn.cursor()
    print("Connected to PostgreSQL successfully.")
except Exception as e:
    print(f"Failed to connect to PostgreSQL: {e}")
    exit()

try:
    print("Starting to consume messages.")
    for message in consumer:
        data = message.value
        print(f"Received message: {data}")

        # Insert data into PostgreSQL
        try:
            columns = data.keys()
            values = [tuple(data.values())]
            insert_query = f"INSERT INTO malware_data ({', '.join(columns)}) VALUES %s"
            execute_values(cursor, insert_query, values)
            conn.commit()
            print("Data inserted successfully")
        except Exception as e:
            print(f"Failed to insert data: {e}")
            conn.rollback()
except KeyboardInterrupt:
    print("Consumer interrupted.")
finally:
    # Close PostgreSQL connection
    if cursor:
        cursor.close()
    if conn:
        conn.close()
        print("PostgreSQL connection closed.")
